In [1]:
import rioxarray as rxr
import os
from rasterio.plot import show
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd

In [2]:
def load_data_from_file(contained_file):
    # load the data
    with open(contained_file) as f:
        lines = f.readlines()
        # reteive line infos
        segment_id = []
        polygon_id = []
        class_id = []
        pixels = []
        for line in lines:
            # retrieve segment id
            segment_id.append(line.split(',')[0].split(':')[1].strip())
            # retrieve polygon id
            polygon_id.append(line.split(',')[1].split(':')[1].strip())
            # retrieve class id
            class_id.append(line.split(',')[2].split(':')[1].strip())
            # retrieve pixels
            pixel_list_str = line.split(':')[4]
            pixel_list_str = pixel_list_str[3:-3]
            pixel_list = list(pixel_list_str.split('], ['))
            pixel_list = [pixel.split(', ') for pixel in pixel_list]
            pixel_list = [[int(pixel[0]), int(pixel[1])] for pixel in pixel_list]
            pixels.append(pixel_list)
        
        # create a dataframe
        df = pd.DataFrame()
        df['segment_id'] = segment_id
        df['polygon_id'] = polygon_id
        df['class_id'] = class_id
        df['pixels'] = pixels
        
        return df

In [3]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

In [4]:
dataframe = load_data_from_file(contained_file_50)

In [5]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb"

list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

In [6]:
def clip_all_acquisition(band, output_folder):

    with rasterio.open(band) as src:
        for acquisition in range(1, src.count):
            print(f"Acquisition {acquisition}/{src.count}")
            data_array = src.read(acquisition)
            min_val = np.percentile(data_array, 2)
            max_val = np.percentile(data_array, 98)
            norm_band = np.clip( (data_array - min_val) / (max_val - min_val), 0, 1)
            
            # Define the output file path for the normalized band
            output_path = os.path.join(output_folder, f'normalized_{os.path.basename(band)}_Acquisition_{acquisition}.tif')

            # Create a new GeoTIFF file and write the normalized data
            with rasterio.open(output_path, 'w', **src.profile) as dst:
                dst.write(norm_band, 1)

In [8]:
for band in list_bands:
    print(f"Band {band}")
    band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')
    output_folder = os.path.join(folder, 'normalized_output')  # Output folder for the normalized bands
    os.makedirs(output_folder, exist_ok=True)
    clip_all_acquisition(band_file, output_folder)

Band 02
Acquisition 1/73


Acquisition 2/73


KeyboardInterrupt: 

In [11]:
file = os.path.join(folder, 'normalized_output/normalized_s2_2020_B02.tif_Acquisition_1.tif')
# show infos on the tif file
with rasterio.open(file) as src:
    print(src.profile)
    print(src.bounds)
    print(src.crs)
    # show shape
    print(src.shape)
    print(src.count)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 5183, 'height': 4513, 'count': 73, 'crs': CRS.from_epsg(32630), 'transform': Affine(10.0, 0.0, 402570.0,
       0.0, -10.0, 1257190.0), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}
BoundingBox(left=402570.0, bottom=1212060.0, right=454400.0, top=1257190.0)
EPSG:32630
(4513, 5183)
73


- Satellite raster data (Sentinel 2)
	- 10 spectral bands, one per file (s2_2020_B02.tif, s2_2020_B03.tif, etc...)
	- Each file s2_2020_BXX.tif: contains an array of size (73, 4513, 5183)=(nb. dates, height, width).

In [ ]:
acquisition = 73
def get_7310(pixel):
    x = pixel[0]
    y = pixel[1]
    pixel_7310 = []
    for band in normalized_data_bands:
        for period in acquisition:
            pixel_7310.append(bands[acquisition, x, y])
    return pixel_7310